# Tugas Event #2 - Solving Real-World Problems with Data Science

*   Nama: Dimas Shidqi Parikesit
*   NIM: 16520105
*   Universitas: Institut Teknologi Bandung
*   Tanggal mulai: 10 November 2020
*   Dataset: https://www.kaggle.com/camnugent/california-housing-prices
*   Disclaimer: This notebook was made by following guide from https://www.kaggle.com/ravichaubey1506/end-to-end-machine-learning/notebook#Model-Tuning


# Housing.csv



### 1.   Data Preaparation & Observation

*   Pada analisis ini saya menggunakan dataset california housing.
*   Dataset ini memberikan data berupa latitude, longitude, house median age dll






In [ ]:
import pandas as pd
import numpy as np

In [ ]:
housing = pd.read_csv('./drive/My Drive/Colab Notebooks/housing.csv')
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe().T

### 2. Data Visualisation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

Dari 9 grafik diatas, dapat kita lihat bahwa median_income, housing_median_age dan median_house_value memiliki batas. Hal tersebut dapat kita lihat dari grafik, dimana nilainya tiba tiba meninggi.Oleh karena itu, kita akan menghilangkan begian tersebut dari dataset yang diperhitungkan.

In [ ]:
# Pertama, kita ingin melihat frekuensi median_income dari tiap rentang jumlah tertentu
fig = plt.figure(dpi=80, figsize=(6,4))
ax = fig.add_axes([1,1,1,1])
ax.set(xlabel='Median Income Class', ylabel='Frequency', title='Distribution of Median Income')
housing['median_income'].hist(color='blue', ax=ax)
plt.show()

In [ ]:
# Karena median_income memiliki batas atas, kita harus memotong bagian tersebut dari grafik kita
# Kemudian bagian yang tidak dipotong akan dikategorikan kedalam kolom baru
housing['income_cat']=pd.cut(housing['median_income'],
                             bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                             labels=[1,2,3,4,5])
housing['income_cat'].value_counts()

In [ ]:
# Kemudian buat grafik distribusi income berdasarkan income category
fig = plt.figure(dpi = 80, figsize = (6,4))
ax = fig.add_axes([1,1,1,1])
ax.set(xlabel = 'Median Income Category',ylabel = 'Frequency',title = 'Distribution of Median Income Category')
housing["income_cat"].hist(color = 'purple',ax=ax)
plt.show()

Memecah data menjadi train dan test set dengan train set sebesar 0.8 dan test set sebesar 0.2
Kita akan menggunakan stratified sampling dengan asumsi median_income adalah parameter yang baik

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# Kita akan menggunakan berbagai metode visualisasi untuk melihat korelasi data yang ada
# Oleh karena itu kita akan buat copy dataset untuk dilihat-lihat
housing = strat_train_set.copy()

In [ ]:
# Kita ingin mengetahui apakah housing price terkait dengan lokasi dan kepadatan penduduk
fig = plt.figure(dpi = 100,figsize = (4,4))
ax = fig.add_axes([1,1,1,1])

import matplotlib.image as mpimg
california_img=mpimg.imread("https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/California_Locator_Map.PNG/280px-California_Locator_Map.PNG")
housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),ax=ax,
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
plt.show();

Ternyata housing price sangat dipengaruhi oleh lokasi dan kepadatan penduduk

In [ ]:
# Kemudian kita ingin mengetahui korelasi antar kolom
import seaborn as sns
corr = housing.corr()
mask = np.triu(np.ones_like(corr,dtype = bool))

plt.figure(dpi=100)
plt.title('Correlation Analysis')
sns.heatmap(corr,mask=mask,annot=False,lw=0,linecolor='white',cmap='viridis',fmt = "0.2f")
plt.xticks(rotation=90)
plt.yticks(rotation = 0)
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

Dari kedua grafik diatas dapat dilihat bahwa attribut yang paling berkorelasi dengan median_house_value adalah median_income.
Kita akan melihat lebih dekat korelasi antara keduanya

In [ ]:
fig = plt.figure(dpi = 80, figsize = (6,4))
ax = fig.add_axes([1,1,1,1])

housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1,color = 'blue',ax=ax)
plt.axis([0, 16, 0, 550000])
plt.show()

Scatterplot ini memperlihatkan bahwa korelasi keduanya kuat, dapat dilihat dengan trend naik serta titik yang tidak terlalu menyebar

Kemudian kita akan membuat kolom baru menggunakan ruangan, kamar tidur, serta populas

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

corr = housing.corr()
mask = np.triu(np.ones_like(corr,dtype = bool))

plt.figure(dpi=100)
plt.title('Correlation Analysis')
sns.heatmap(corr,mask=mask,annot=False,lw=0,linecolor='white',cmap='magma',fmt = "0.2f")
plt.xticks(rotation=90)
plt.yticks(rotation = 0)
plt.show()

Kolom bedrooms_per_room memiliki korelasi yang lebih baik dengan median house value dibandingkan total number of rooms /bedrooms

### 3. Model Development

#### Data Pipeline

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

In [ ]:
from sklearn.compose import ColumnTransformer

housing_num = housing.drop("ocean_proximity", axis=1)

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

#### Model Training

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=5, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=5, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
print("RMSE ==> ", forest_rmse)

RMSE ==>  25756.027706421155


#### Model Tuning

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [54607.69682829 55676.51534528 56488.93565291 53646.91831517
 56761.83802382]
Mean: 55436.38083309248
Standard deviation: 1167.1560546879546


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

#### Evaluate Model

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
print("RMSE on Test ==> ",final_rmse)

Menghitung 95% confidence interval untuk generalisasi error untuk mengetahui seberapa presisi estimasi ini.

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

### Insight

* Faktor yang banyak mempengaruhi median house value adalah median income
* Median house age memiliki korelasi negatif dengan total ruangan, kamar tidur, serta populasi. Hal ini menandakan bahwa parameter tersebut tidak saling mempengaruhi
* House value di California pada tahun ini memiliki nilai minimum 14999, rata-rata 206855.8 , sedangkan nilai maksimum tidak diketahui dalam data ini karena dibatasi maksimal 500000